In [ ]:
#import mesma 
import rasterio
import matplotlib.pyplot as plt
import os
import numpy as np
import sys
import cupy as cp
import random
from scipy.ndimage import binary_dilation, uniform_filter


In [ ]:
path = r'/mnt/c/Users/attic/HLS_Kelp/imagery/rf_classified_cuML/HLS.L30.T11SKU.2017327T183418.v2.0_kelp_classified.tif'
ocean_dilation = np.ones((100,100)) #Struct for dilation (increase to enlarge non-ocean mask) larger --> takes longer
kelp_neighborhood_min = 5
min_kelp_count = 4

kelp_dilation = np.ones((20,20))

with rasterio.open(path) as imagery:
    classified_img = imagery.read(5)
    kelp_mask  = []
    ocean_mask = []
    land_dilated = np.where(classified_img == 3, True, False)
    clouds_dilated = np.where(classified_img == 2, True, False)
    land_dilated = binary_dilation(land_dilated, structure=ocean_dilation)
    #ocean_dilated = np.where(classified_img == 1, True, False)
    #ocean_dilated = binary_dilation(ocean_dilated, structure=ocean_dilation) #This takes ~25 seconds. Should look to optimize
    ocean_dilated = land_dilated | clouds_dilated 
    #
    kelp_dilated = np.where(classified_img == 0, True, False) #This is expanding hte kelp_mask so the TF is reversed
    kelp_count = uniform_filter(kelp_dilated.astype(int),size=kelp_neighborhood_min)
    kelp_dilated = np.where(((~kelp_dilated) | (kelp_count >= min_kelp_count)), 0, 1) #If theres no kelp, or the kelp count is <=4, set pixel == false
    kelp_dilated = binary_dilation(kelp_dilated,structure=kelp_dilation) #I may not want to do this. we'll see
    for i in range(4):
        band_data = imagery.read(i + 1)
        kmask = np.where(kelp_dilated == True, band_data, np.nan)
        omask = np.where((ocean_dilated == False), band_data, np.nan)
        kelp_mask.append(kmask)
        ocean_mask.append(omask)

    kelp_mask = np.array(kelp_mask)
    ocean_mask = np.array(ocean_mask)
    #print(ocean_mask)

rgb_nor = np.stack([ocean_mask[2]/600,ocean_mask[0]/600,ocean_mask[1]/600], axis=-1)
rgb_nor_cropped = rgb_nor
#print(kelp_mask)
rgb_nor_cropped = np.ma.masked_where(np.isnan(rgb_nor_cropped), rgb_nor_cropped)
image = kelp_mask[1]#,2500:3500,800:1800]
plt.figure(figsize=(30, 30), dpi=200)
plt.imshow(image, alpha=1)
plt.imshow(rgb_nor_cropped, alpha=1)
plt.colorbar()
plt.show()

In [ ]:
ocean_EM_stack = []
kelp_EM = [459, 556, 437, 1227]
n_bands, height, width = kelp_mask.shape
ocean_EM_n = 0
ocean_data = ocean_mask.reshape(ocean_mask.shape[0], -1)
kelp_data = kelp_mask.reshape(kelp_mask.shape[0],-1)

nan_columns = np.isnan(ocean_data).any(axis=0)
# Remove columns with NaN values
filtered_ocean = ocean_data[:, ~nan_columns]
print(filtered_ocean.shape)
if(len(filtered_ocean[0,:]) < 1000):
     print("Insufficient number of ocean pixels")
     sys.exit()
for i in range(30):
     index = random.randint(0,len(filtered_ocean[0])-1)
     ocean_EM_stack.append(filtered_ocean[:,index])
ocean_EM = np.stack(ocean_EM_stack, axis=1)
#print(ocean_EM_array)


average_val = np.nanmean(filtered_ocean, axis=1)
average_endmember = np.nanmean(ocean_EM, axis=1)
print(f"average EM Val: {average_endmember}")
print(f"average    Val: {average_val}")

In [ ]:
kelp_mask = cp.asarray(kelp_mask)
ocean_EM = cp.asarray(ocean_EM)
kelp_EM = cp.asarray(kelp_EM)
kelp_data = cp.asarray(kelp_data)

frac1 = cp.full((kelp_mask.shape[1], kelp_mask.shape[2], 30), cp.nan)
frac2 = cp.full((kelp_mask.shape[1], kelp_mask.shape[2], 30), cp.nan)
rmse = cp.full((kelp_mask.shape[1], kelp_mask.shape[2], 30), cp.nan)
print(rmse.shape)

#print("Running MESMA")
for k in range(30):
    B = cp.column_stack((ocean_EM[:, k], kelp_EM))
    U, S, Vt = cp.linalg.svd(B, full_matrices=False)
    IS = Vt.T / S
    em_inv = IS @ U.T
    F = em_inv @ kelp_data
    model = (F.T @ B.T).T
    resids = (kelp_data - model) / 10000
    rmse[:, :, k] = cp.sqrt(cp.mean(resids**2, axis=0)).reshape(height, width)
    frac1[:, :, k] = F[0, :].reshape(height, width)
    frac2[:, :, k] = F[1, :].reshape(height, width)
    #print(f'Percent MESMA {round(100/30 * (k+1))}%')

In [ ]:
minVals = cp.nanmin(rmse, axis=2)
PageIdx = cp.nanargmin(rmse, axis=2)

rows, cols = cp.meshgrid(cp.arange(rmse.shape[0]), cp.arange(rmse.shape[1]), indexing='ij')

Zindex = cp.ravel_multi_index((rows, cols, PageIdx), dims=rmse.shape)

Mes2 = frac2.ravel()[Zindex]
Mes2 = Mes2.T
Mes2 = -0.229 * Mes2**2 + 1.449 * Mes2 - 0.018 #Landsat mesma corrections 
Mes2 = cp.clip(Mes2, 0, None)  # Ensure no negative values
Mes2 = cp.round(Mes2 * 100).astype(cp.int16)


In [ ]:
#Mes2 = Mes2.astype(cp.float32)
#Mes2 = Mes2.where(Mes2 == 0, cp.nan)
kelp_img = cp.asnumpy(kelp_mask)
Mes_array = cp.asnumpy(Mes2).T
Mes_array = np.where(Mes_array == 0, np.nan, Mes_array)
plt.figure(figsize=(20, 20), dpi=200)
plt.imshow(kelp_img[1,2800:3100,800:1400] , cmap='Greys')
plt.imshow(Mes_array[2800:3100,800:1400], alpha=1)
plt.colorbar()
plt.show()